### Create dictionary

In [1]:
from counter import doc_iterator
from gensim import corpora
import gensim
from gensim.parsing.preprocessing import remove_stopwords
import pickle
import nltk
import regex as re
from pprint import pprint

In [2]:
jstor_zip_base = "C:/Users/benja/OneDrive - Cornell University/Desktop/soc_econ/data"

In [3]:
def get_di():
    return doc_iterator(
        jstor_zip_base,
        SKIP_N=99, # samples from entire corpus, takes 1 out of 100 documents
    )

In [4]:
def text_iterator(it):
    for i,(doi, doc) in enumerate(it):
        if i%1000 == 0:
            print(f"Document {i}")
        # if i == 0: print(doc['content'])
        yield [item for item in re.findall( "[A-Za-z]+", remove_stopwords(doc['content'].lower()) ) if len(item)>3] # yield function only temporarily loads the documents into storage

In [5]:
di = get_di()

In [6]:
dictionary = corpora.Dictionary( text_iterator(di) )

Found files receipt-id-1215891-part-001.zip, receipt-id-1383671-part-001.zip, receipt-id-1383681-part-001.zip, receipt-id-1383691-part-001.zip
Iterating over  248380 documents
Document 0


In [7]:
outfile = open('gensimdic.pkl', 'wb')
pickle.dump(dictionary, outfile)                     
outfile.close()

### Run LDA

In [8]:
infile = open('gensimdic.pkl','rb')
dictionary = pickle.load(infile)
infile.close()

In [9]:
class MyCorpus(object):
    def __init__(self):
        self.di = get_di()        
        self.ti = text_iterator(self.di)
    def __iter__(self):
        yield dictionary.doc2bow( next(self.ti) )

In [16]:
# Run LDA by document and update the parameters
corpus = MyCorpus()
lda = gensim.models.ldamodel.LdaModel(corpus=corpus, 
                                      id2word=dictionary,
                                      num_topics=10,
                                      update_every=10,
                                      chunksize=10000,
                                      passes=100)

Found files receipt-id-1215891-part-001.zip, receipt-id-1383671-part-001.zip, receipt-id-1383681-part-001.zip, receipt-id-1383691-part-001.zip
Iterating over  248380 documents
Document 0


In [17]:
for topic_id in range(lda.num_topics):
    topk = lda.show_topic(topic_id, 10)
    topk_words = [ w for w, _ in topk ]
    
    print('{}: {}'.format(topic_id, ' '.join(topk_words)))

0: girls school education year schools academic encouragement social friends australia
1: girls school schools social education academic encouragement composition year queensland
2: girls schools school academic education year social coeducational encouragement composition
3: children social attitudes child marriage family behavior group parents status
4: girls school education schools social academic year queensland composition australian
5: girls school schools social education academic year students achievement encouragement
6: girls school schools education academic year social australian achievement composition
7: girls school education schools social academic students year encouragement composition
8: girls palestinian education achievements schools abortion cohabitation college private social
9: schools girls school education social year academic australian private queensland


In [15]:
?gensim.models.ldamodel.LdaModel